In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
from sklearn.metrics.pairwise import cosine_similarity


tfidf_matrix = load_npz("tfidf_matrix.npz")
df_wisata = pd.read_csv("../0. Dataset/Dataset_Toba_Text_Processed.csv")
df_history = pd.read_csv("../0. Dataset/userHistory.csv")

# === Fungsi untuk rekomendasi berdasarkan user_id ===
def recommend_for_user(user_id, top_n=5):
    # Ambil nama wisata yang dikunjungi user
    user_history = df_history[df_history['userID'] == user_id]['namaWisata'].tolist()
    
    # Ambil indeks wisata yang cocok di data utama
    visited_indices = df_wisata[df_wisata['title'].isin(user_history)].index

    if len(visited_indices) == 0:
        return f"Tidak ada data wisata cocok di history user {user_id}."

    # Bangun profil user dari rata-rata TF-IDF wisata yang dikunjungi
    user_profile_matrix = tfidf_matrix[visited_indices].mean(axis=0)
    user_profile = np.asarray(user_profile_matrix).reshape(1, -1)

    # Hitung similarity
    similarities = cosine_similarity(user_profile, tfidf_matrix).flatten()

    # tambah ke dataframe
    df_wisata['similarity'] = similarities

    # Bikin rekomendasi, kecuali yang sudah dikunjungi
    rekomendasi = df_wisata[~df_wisata['title'].isin(user_history)]
    rekomendasi = rekomendasi.sort_values(by='similarity', ascending=False)

    return rekomendasi[['title', 'similarity']].head(top_n).to_numpy(), user_history

# === Contoh penggunaan ===
if __name__ == "__main__":
    user_id = 2  # Ganti sesuai kebutuhan (PAKAI API)
    result, user_history= recommend_for_user(user_id)
    print(f'User dengan id = {user_id} Pernah mengunjungi: ')
    for history in user_history:
        print(f'- {history}')

    print(f'\n Wisata Yang Direkomendasikan Untuk User id = {user_id}')
    for judul, skor in result:
        print(f'* {judul} {skor:.2f}')


User dengan id = 2 Pernah mengunjungi: 
- Pantai Batuhoda
- Pantai Sipinggan
- Bukit Simargulang Ombun
- Bukit Gajah Bobok
- Bukit Sibeabea

 Wisata Yang Direkomendasikan Untuk User id = 2
* Pantai Indah Situngkir (PIS) 0.38
* Bukit Senyum 0.38
* Pantai Ikan Mas Tandarabun 0.36
* Bukit Beta Tuk-tuk 0.36
* Bukit Holbung Samosir 0.34
